#### AI for Retail & Consumer Goods | Group Project | MBD April 2024

Group Members: Maud Lecerf | Cristina Mosquera | Christopher Stephan

# Base Imports 

In [32]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

In [35]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [42]:
# Custom Theme
iowa = {
    "layout": {
        "colorway": ["#4c3624", "#ff0651", "#ff7e06", "#74B36B", "#628395",
                     "#996C48", "#FF5C8D", "#FF9633", "#8CC084", "#84A0AE",
                     "#B78B66", "#FF85A9", "#FFB570", "#A7CEA1", "#A9BCC7"], 
        "plot_bgcolor": "white",
        "paper_bgcolor": "white",
        "font": {"color": "black"},
        "xaxis": {"gridcolor": "lightgray"},
        "yaxis": {"gridcolor": "lightgray"},
    }
}

# Register themes
pio.templates["iowa"] = iowa

In [2]:
daclean = pd.read_csv("G:\\Mon Drive\\RCG\\Iowa_Liquor_Sales_Cleaned.csv")
data_post_covid = pd.read_csv("G:\\Mon Drive\\RCG\\Iowa_Liquor_Sales_Post_Covid.csv")

#data = pd.read_csv("/Users/christopherstephan/Documents/IE/Term 3/Consumer Goods/project/Iowa_Liquor_Sales_DEC24.csv") #Chris

C:\Users\ginny\AppData\Local\Temp\ipykernel_28444\2207171519.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  daclean = pd.read_csv("G:\\Mon Drive\\RCG\\Iowa_Liquor_Sales_Cleaned.csv")


In [3]:
daclean.head()

,invoice_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons)
0,S03408100057,2012-01-03,3162,NASH FINCH / WHOLESALE FOOD,807 GRANDVIEW,MUSCATINE,52761,POINT (-91.06411300000002 41.408437),70.0,MUSCATINE,...,43127,BACARDI SUPERIOR RUM,12,1000,9.43,14.14,4,56.56,4.00,1.06
1,S03408800027,2012-01-03,2959,DAHL'S / MERLE HAY,4343 MERLE HAY ROAD,DES MOINES,50310,POINT (-93.697647 41.63740800000001),77.0,POLK,...,36188,GORDON'S VODKA 80 PRF - PET,6,1750,9.68,14.52,6,87.12,10.50,2.77
2,S03382600002,2012-01-03,4317,FAREWAY STORES #941 / GREENFIELD,212 SW KENT,GREENFIELD,50849,POINT (-94.462719 41.303167),1.0,ADAIR,...,11296,CROWN ROYAL,12,750,14.25,21.37,12,256.44,9.00,2.38
3,S03411200041,2012-01-03,2587,HY-VEE FOOD STORE / JOHNSTON,5750 MERLE HAY ROAD,JOHNSTON,50131,POINT (-93.697731 41.665172),77.0,POLK,...,89786,SAUZA GOLD,12,750,8.40,12.59,1,12.59,0.75,0.20
4,S03410100024,2012-01-03,3962,DURANT STREET COUNTRY STOP,618 DURANT ST,HARLAN,51537,POINT (-95.318597 41.656803),83.0,SHELBY,...,65061,GOLDSCHLAGER CINNAMON SCHNAPPS MINI,10,600,13.73,20.60,1,20.60,0.60,0.16


In [4]:
numerical_cols = ['pack', 'bottle_volume_(ml)', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_(dollars)',
                  'volume_sold_(liters)', 'volume_sold_(gallons)' ]
categorical_cols = ['invoice_item_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'store_location', 'county_number', 'county',
       'category', 'category_name', 'vendor_number', 'vendor_name',
       'item_number', 'item_description' ]

In [5]:
daclean.columns

Index(['invoice_item_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'store_location', 'county_number', 'county',
       'category', 'category_name', 'vendor_number', 'vendor_name',
       'item_number', 'item_description', 'pack', 'bottle_volume_(ml)',
       'state_bottle_cost', 'state_bottle_retail', 'bottles_sold',
       'sale_(dollars)', 'volume_sold_(liters)', 'volume_sold_(gallons)'],
      dtype='object')

In [6]:
daclean[numerical_cols].describe()

,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons)
count,2.969686e+07,2.969686e+07,2.969686e+07,2.969686e+07,2.969686e+07,2.969686e+07,2.969686e+07,2.969686e+07
mean,1.211928e+01,8.754152e+02,1.076429e+01,1.615636e+01,1.085935e+01,1.457653e+02,9.149051e+00,2.414079e+00
std,7.782577e+00,6.237240e+02,1.326406e+01,1.989482e+01,3.060839e+01,5.144071e+02,3.633208e+01,9.598031e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.680000e+02,-9.720000e+03,-1.344000e+03,-3.550400e+02
25%,6.000000e+00,7.500000e+02,5.740000e+00,8.610000e+00,3.000000e+00,3.597000e+01,1.500000e+00,4.000000e-01
50%,1.200000e+01,7.500000e+02,8.500000e+00,1.275000e+01,6.000000e+00,7.740000e+01,4.800000e+00,1.260000e+00
75%,1.200000e+01,1.000000e+03,1.300000e+01,1.950000e+01,1.200000e+01,1.493000e+02,1.050000e+01,2.770000e+00
max,3.360000e+02,3.780000e+05,2.498902e+04,3.748353e+04,1.500000e+04,2.795573e+05,1.500000e+04,3.962580e+03


In [7]:
daclean.nunique()

invoice_item_number      29696865
date                         3310
store_number                 3124
store_name                   3315
address                      3095
city                          494
zip_code                     1378
store_location               9830
county_number                  99
county                         99
category                      111
category_name                 103
vendor_number                 494
vendor_name                   613
item_number                 13766
item_description            12717
pack                           27
bottle_volume_(ml)             58
state_bottle_cost            3672
state_bottle_retail          4149
bottles_sold                  761
sale_(dollars)              35082
volume_sold_(liters)         1737
volume_sold_(gallons)        2050
dtype: int64

In [8]:
daclean['pack'].unique()

array([ 12,   6,  10,  24,  48,   5,   8,   4,   1,   3,  20,   2,  60,
       120,  30,  16,  18,   9,  32, 160, 288,  44,  96,  15, 336, 312,
        28], dtype=int64)

In [9]:
daclean['bottle_volume_(ml)'].unique()

array([  1000,   1750,    750,    600,    375,    500,    200,   1200,
         2400,    800,    150,    100,    300,   3000,   2000,    250,
          301,    502,     50,    603,     12,   1500,    900,    400,
         1508,   4800,   7580,    850,   1125,      0, 140000,   1950,
           15,   1850,    950,   2250,   4500,   2550,   3600,   6000,
       180000, 225000, 189000,   9000,   3900,   3500, 378000,   1800,
        31500,     20,    425,   2125,    355,   1420,     25,    175,
         5250,    700], dtype=int64)

In [10]:
daclean['category_name'].unique()

array(['PUERTO RICO & VIRGIN ISLANDS RUM', 'VODKA 80 PROOF',
       'CANADIAN WHISKIES', 'TEQUILA', 'IMPORTED SCHNAPPS',
       'ROOT BEER SCHNAPPS', 'STRAIGHT BOURBON WHISKIES',
       'AMERICAN GRAPE BRANDIES', 'IMPORTED VODKAS',
       'BUTTERSCOTCH SCHNAPPS', 'MISC. IMPORTED CORDIALS & LIQUEURS',
       'AMERICAN COCKTAILS', 'BLACKBERRY BRANDIES', 'TENNESSEE WHISKIES',
       'WATERMELON SCHNAPPS', 'CREAM LIQUEURS', 'VODKA FLAVORED',
       'IMPORTED VODKA - MISC', 'COFFEE LIQUEURS', 'BLENDED WHISKIES',
       'STRAWBERRY SCHNAPPS', 'SPICED RUM', 'AMERICAN DRY GINS',
       'PEPPERMINT SCHNAPPS', '100 PROOF VODKA', 'FLAVORED RUM',
       'SCOTCH WHISKIES', 'PEACH SCHNAPPS', 'JAMAICA RUM',
       'MISC. AMERICAN CORDIALS & LIQUEURS', 'AMERICAN AMARETTO',
       'IRISH WHISKIES', 'SINGLE MALT SCOTCH', 'TRIPLE SEC',
       'CINNAMON SCHNAPPS', 'IMPORTED GRAPE BRANDIES',
       'MISCELLANEOUS SCHNAPPS', 'CHERRY BRANDIES', 'AMERICAN ALCOHOL',
       'APPLE SCHNAPPS', 'WHISKEY LIQUEUR', 

In [11]:
daclean['county'].unique()

array(['MUSCATINE', 'POLK', 'ADAIR', 'SHELBY', 'LEE', 'POTTAWATTAMIE',
       'DICKINSON', 'SCOTT', 'DES MOINES', 'JONES', 'VAN BUREN',
       'MONTGOMERY', 'BREMER', 'DUBUQUE', 'WRIGHT', 'CLINTON', 'WAYNE',
       'CERRO GORDO', 'CARROLL', 'WINNEBAGO', 'EMMET', 'JACKSON', 'PAGE',
       'IOWA', 'DAVIS', 'BUTLER', 'WARREN', 'CASS', 'STORY', 'ADAMS',
       'CLAY', 'FRANKLIN', 'TAYLOR', 'WASHINGTON', 'BLACK HAWK', 'HARDIN',
       'FAYETTE', 'AUDUBON', 'LINN', 'GUTHRIE', 'GRUNDY', 'MARSHALL',
       'MILLS', 'CLAYTON', 'JOHNSON', 'KOSSUTH', 'DELAWARE', 'HAMILTON',
       'BENTON', "O'BRIEN", 'HENRY', 'WAPELLO', 'BUCHANAN', 'KEOKUK',
       'LYON', 'CALHOUN', 'PALO ALTO', 'CRAWFORD', 'MARION', 'IDA',
       'WEBSTER', 'OSCEOLA', 'WOODBURY', 'BUENA VISTA', 'POCAHONTAS',
       'APPANOOSE', 'HARRISON', 'MAHASKA', 'HANCOCK', 'MONONA',
       'JEFFERSON', 'SAC', 'MONROE', 'HUMBOLDT', 'UNION', 'TAMA',
       'LOUISA', 'DECATUR', 'POWESHIEK', 'JASPER', 'DALLAS', 'SIOUX',
       'CLARKE', 'CEDA

In [12]:
# Define mapping of keywords to subcategories
category_mapping = {
    'RUM': 'RUM',
    'VODKA': 'VODKAS',
    'VODKAS': 'VODKAS',
    'SCHNAPPS': 'SCHNAPPS',
    'BRANDIES': 'BRANDIES',
    'WHISKIES': 'WHISKIES',
    'SCOTCH': 'WHISKIES',
    'GINS': 'GINS',
    'GIN': 'GINS',
    'MEZCAL': 'MEZCAL',
    'TEQUILA': 'TEQUILA',
    'BOURBON': 'BOURBON',
    'LIQUEURS': 'LIQUEURS',
    'LIQUEUR':'LIQUEURS',
    'AMARETTO': 'AMARETTO',
    'CREME': 'CREME'
}

# Create a new column and assign "OTHERS" by default
daclean['subcategory'] = 'OTHERS'

# Loop through mapping and assign subcategories
for keyword, subcategory in category_mapping.items():
    daclean.loc[daclean['category_name'].str.contains(keyword, case=False, na=False, regex=True), 'subcategory'] = subcategory



In [13]:
daclean['subcategory'].value_counts()

subcategory
VODKAS      7435198
WHISKIES    6202828
LIQUEURS    3531768
RUM         2783913
BOURBON     2005501
OTHERS      1769648
TEQUILA     1561181
SCHNAPPS    1475144
BRANDIES    1468884
GINS        1354631
AMARETTO      65967
CREME         26684
MEZCAL        15518
Name: count, dtype: int64

In [14]:
daclean[daclean['subcategory'] == 'OTHERS']['category_name'].unique()

array(['AMERICAN COCKTAILS', 'TRIPLE SEC', 'AMERICAN ALCOHOL', 'ANISETTE',
       'DISTILLED SPIRITS SPECIALTY', 'ROCK & RYE',
       'DECANTERS & SPECIALTY PACKAGES', 'HIGH PROOF BEER - AMERICAN',
       'JAPANESE WHISKY', 'COCKTAILS/RTD',
       'IMPORTED DISTILLED SPIRITS SPECIALTY', 'NEUTRAL GRAIN SPIRITS',
       'AMERICAN DISTILLED SPIRITS SPECIALTY', 'DELISTED ITEMS',
       'NEUTRAL GRAIN SPIRITS FLAVORED', 'SPECIAL ORDER ITEMS',
       'IOWA DISTILLERIES', 'TEMPORARY & SPECIALTY PACKAGES',
       'HOLIDAY VAP', 'DELISTED / SPECIAL ORDER ITEMS'], dtype=object)

In [16]:
daclean.dtypes

invoice_item_number       object
date                      object
store_number               int64
store_name                object
address                   object
city                      object
zip_code                  object
store_location            object
county_number            float64
county                    object
category                 float64
category_name             object
vendor_number            float64
vendor_name               object
item_number                int64
item_description          object
pack                       int64
bottle_volume_(ml)         int64
state_bottle_cost        float64
state_bottle_retail      float64
bottles_sold               int64
sale_(dollars)           float64
volume_sold_(liters)     float64
volume_sold_(gallons)    float64
subcategory               object
dtype: object

## Time-based analysis 

In [17]:
daclean['date'] = pd.to_datetime(daclean['date'])

In [18]:
data_dt = daclean.copy()

In [19]:
#removing all columns that are repetition of another column or columns from which we won't be able to extract any insights
data_dt = data_dt.drop(columns=['county_number', 'vendor_number', 'store_number', 'invoice_item_number'])

In [20]:
data_dt['year'] = data_dt['date'].dt.year
data_dt['month'] = data_dt['date'].dt.month
data_dt['day'] = data_dt['date'].dt.day
data_dt['day_of_week'] = data_dt['date'].dt.day_name()

In [22]:
#columns that we can aggregate by date to further our analysis 
agg_columns = ['state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 
               'sale_(dollars)', 'volume_sold_(liters)', 'volume_sold_(gallons)']

In [25]:
# Group by date
grouped_by_date = data_dt.groupby('date')[agg_columns].sum().reset_index()
grouped_by_date.set_index('date', inplace=True)

# Group by year
grouped_by_year = data_dt.groupby('year')[agg_columns].sum().reset_index()

# Group by month
grouped_by_month = data_dt.groupby('month')[agg_columns].sum().reset_index()

# Group by day
grouped_by_day = data_dt.groupby('day')[agg_columns].sum().reset_index()

# Group by day of the week
grouped_by_weekday = data_dt.groupby('day_of_week')[agg_columns].sum().reset_index()

In [26]:
# Group by date and subcategory
grouped_by_date_subcat = data_dt.groupby(['date', 'subcategory'])[agg_columns].sum().reset_index()
grouped_by_date_subcat.set_index('date', inplace=True)

# Group by year and subcategory
grouped_by_year_subcat = data_dt.groupby(['year', 'subcategory'])[agg_columns].sum().reset_index()

# Group by month and subcategory
grouped_by_month_subcat = data_dt.groupby(['month', 'subcategory'])[agg_columns].sum().reset_index()

# Group by day and subcategory
grouped_by_day_subcat = data_dt.groupby(['day', 'subcategory'])[agg_columns].sum().reset_index()

# Group by day of the week and subcategory
grouped_by_weekday_subcat = data_dt.groupby(['day_of_week', 'subcategory'])[agg_columns].sum().reset_index()

In [37]:
px.line(grouped_by_date, y='bottles_sold', template= 'iowa', title= 'Bottles sold through time')

In [46]:
px.bar(grouped_by_year_subcat, x= 'year', y='bottles_sold', color='subcategory', template = 'iowa', 
       title = 'Bottles Sold every year by Alcohol Type')

In [48]:
px.bar(grouped_by_year_subcat, x= 'year', y='volume_sold_(liters)', color='subcategory', template = 'iowa', 
       title = 'Bottles Sold every year by Alcohol Type')

In [49]:
px.bar(grouped_by_year_subcat, x= 'year', y='sale_(dollars)', color='subcategory', template = 'iowa', 
       title = 'Bottles Sold every year by Alcohol Type')

## Alcohol Sales by Category 

Let's look deeper into the alcohol sales by category 

In [47]:
agg_columns

['state_bottle_cost',
 'state_bottle_retail',
 'bottles_sold',
 'sale_(dollars)',
 'volume_sold_(liters)',
 'volume_sold_(gallons)']